# MNL model estimations
### 3 main resultant models below:
### [01] - general MNL
### [02] - MNL for time-critical trips
### [03] - MNL for non-time-critical trips

In [40]:
from headers import *

In [41]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import numpy as np

### 1. import 'raw' survey results (full spreadsheet) 

In [42]:
df_full = pd.read_excel("input_df_ENG.xlsx")
df_full.index.name = "index"
df_full.head(10)

,ID,Timestamp,Q1_general_impact,O2a_denied_boading,Q2b_overcrowding,Q2c_moderate_crowding,Q2d_seats_possible,Q2e_empty,Q3_trip_purpose,Q4_time_criticality,...,Q8_expectations,Q9a_SP_3-2_10mins,Q9b_SP_3-2_5mins,Q10a_SP_4-3_10mins,Q10b_SP_4-3_5mins,Q11a_SP_4-2_10mins,Q11b_SP_4-2_5mins,Q12_RTCI_needed,Q13_age,Q14_gender
index,,,,,,,,,,,,,,,,,,,,,
0,1,2019-03-18 08:07:44.810,[3] sometimes,[1] NEVER,[3] sometimes,[4] often,[4] often,[2] rarely,[H] -> other,1,...,[d] will get a seat,0,5,0,5,0,5,4.0,[>65],[F]
1,2,2019-03-18 08:33:21.627,[2] rarely,[1] NEVER,[2] rarely,[4] often,[3] sometimes,[2] rarely,[H] -> work,1,...,[c] will stand comfortably,0,0,0,0,0,0,3.0,[41 - 50],[M]
2,3,2019-03-18 08:40:50.041,[4] often,[1] NEVER,[4] often,[4] often,[3] sometimes,[2] rarely,[H] -> work,1,...,[c] will stand comfortably,0,0,0,5,0,5,5.0,[26 - 40],[F]
3,4,2019-03-18 08:45:58.288,[3] sometimes,[2] rarely,[1] NEVER,[4] often,[4] often,[3] sometimes,[H] -> education,1,...,[c] will stand comfortably,0,0,0,5,0,5,4.0,[18 - 25],[M]
4,5,2019-03-18 08:52:57.112,[4] often,[3] sometimes,[4] often,[2] rarely,[2] rarely,[4] often,[H] -> work,0,...,[b] will have to stand uncomfortably,10,5,10,5,10,5,5.0,[26 - 40],[M]
5,6,2019-03-18 08:57:06.268,[4] often,[2] rarely,[4] often,[2] rarely,[2] rarely,[2] rarely,[H] -> education,1,...,[b] will have to stand uncomfortably,0,0,10,5,10,5,4.0,[18 - 25],[M]
6,7,2019-03-18 09:05:38.328,[4] often,[2] rarely,[3] sometimes,[3] sometimes,[4] often,[2] rarely,[H] -> work,0,...,[c] will stand comfortably,10,5,10,5,10,5,4.0,[18 - 25],[M]
7,8,2019-03-18 09:10:47.972,[4] often,[2] rarely,[4] often,[3] sometimes,[3] sometimes,[2] rarely,[H] -> work,1,...,[c] will stand comfortably,0,0,0,5,0,5,5.0,[18 - 25],[F]
8,9,2019-03-18 09:14:40.524,[3] sometimes,[2] rarely,[2] rarely,[3] sometimes,[2] rarely,[2] rarely,[H] -> education,1,...,[c] will stand comfortably,0,0,10,5,10,5,4.0,[18 - 25],[M]


### 2. process input dataframe 
### i.e. filter and prepare only attributes relevant for BIOGEME estimation

In [43]:
df = df_full[['ID','Q4_time_criticality','Q5_trip_frequency','Q6_journey_time','Q9a_SP_3-2_10mins','Q9b_SP_3-2_5mins','Q10a_SP_4-3_10mins', 'Q10b_SP_4-3_5mins', 'Q11a_SP_4-2_10mins',
       'Q11b_SP_4-2_5mins','Q13_age']]

df_scenario_1 = df.iloc[:,np.r_[0:4,4,10]]
df_scenario_1['scenario'] = 1
df_scenario_1['WT'] = 10
df_scenario_1['output_choice'] = [1 if x == 10 else 0 for x in df_scenario_1['Q9a_SP_3-2_10mins']]
df_scenario_1.drop('Q9a_SP_3-2_10mins', axis = 1, inplace = True)

df_scenario_2 = df.iloc[:,np.r_[0:4,5,10]]
df_scenario_2['scenario'] = 2
df_scenario_2['WT'] = 5
df_scenario_2['output_choice'] = [1 if x == 5 else 0 for x in df_scenario_2['Q9b_SP_3-2_5mins']]
df_scenario_2.drop('Q9b_SP_3-2_5mins', axis = 1, inplace = True)

df_scenario_3 = df.iloc[:,np.r_[0:4,6,10]]
df_scenario_3['scenario'] = 3
df_scenario_3['WT'] = 10
df_scenario_3['output_choice'] = [1 if x == 10 else 0 for x in df_scenario_3['Q10a_SP_4-3_10mins']]
df_scenario_3.drop('Q10a_SP_4-3_10mins', axis = 1, inplace = True)

df_scenario_4 = df.iloc[:,np.r_[0:4,7,10]]
df_scenario_4['scenario'] = 4
df_scenario_4['WT'] = 5
df_scenario_4['output_choice'] = [1 if x == 5 else 0 for x in df_scenario_4['Q10b_SP_4-3_5mins']]
df_scenario_4.drop('Q10b_SP_4-3_5mins', axis = 1, inplace = True)

df_scenario_5 = df.iloc[:,np.r_[0:4,8,10]]
df_scenario_5['scenario'] = 5
df_scenario_5['WT'] = 10
df_scenario_5['output_choice'] = [1 if x == 10 else 0 for x in df_scenario_5['Q11a_SP_4-2_10mins']]
df_scenario_5.drop('Q11a_SP_4-2_10mins', axis = 1, inplace = True)

df_scenario_6 = df.iloc[:,np.r_[0:4,9,10]]
df_scenario_6['scenario'] = 6
df_scenario_6['WT'] = 5
df_scenario_6['output_choice'] = [1 if x == 5 else 0 for x in df_scenario_6['Q11b_SP_4-2_5mins']]
df_scenario_6.drop('Q11b_SP_4-2_5mins', axis = 1, inplace = True)

df_analysis = pd.concat([df_scenario_1, df_scenario_2, df_scenario_3, df_scenario_4, df_scenario_5, df_scenario_6])
df_analysis['age_65plus'] = [1 if x == '[>65]' else 0 for x in df_analysis['Q13_age']]
df_analysis['age_50_65'] = [1 if x == '[51 - 65]' else 0 for x in df_analysis['Q13_age']]
df_analysis['commuter'] = [1 if ((x == '[5-7] days per week') | (x == '[2-4] days per week')) else 0 for x in df_analysis['Q5_trip_frequency']]
df_analysis.drop('Q13_age', axis = 1, inplace = True)
df_analysis.drop('Q5_trip_frequency', axis = 1, inplace = True)
df_analysis.rename(columns = {'ID': 'pass_ID','Q4_time_criticality': 'time_crit', 'Q6_journey_time': 'JRT'}, inplace = True)

df_analysis['Crowding_3_2'] = [1 if ((x == 1) | (x == 2)) else 0 for x in df_analysis['scenario']]
df_analysis['Crowding_4_3'] = [1 if ((x == 3) | (x == 4)) else 0 for x in df_analysis['scenario']]
df_analysis['Crowding_4_2'] = [1 if ((x == 5) | (x == 6)) else 0 for x in df_analysis['scenario']]

columns_order = ['pass_ID','scenario', 'time_crit', 'JRT', 'WT', 'commuter', 'age_50_65', 'age_65plus', 'Crowding_3_2', 'Crowding_4_3', 'Crowding_4_2', 'output_choice']
df_analysis = df_analysis[columns_order]
df_analysis.sort_values(by = ['pass_ID', 'scenario'], inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

## dataframe ready for BIOGEME estimation - attributes:
#### [pass_id] - passenger id
#### [scenario] - i.e. 6 scenarios for the SP experiment: scenario 1 - question 9a, scenario 2 - question 9b, scenario 3 - question 10a etc...
#### [time_crit] - time-criticality of the journey
#### [JRT] - journey time in mins (as stated by passenger)
#### [WT] - waiting time in mins
#### [commuter] - uses this PT route frequently, i.e. at least 2 days per week
#### [age_50-65]
#### [age_65plus]
#### [Crowding_3-2] - RTCI ratio between 1st vs. 2nd run (equal 1 for scenarios 1 and 2, 0 otherwise)
#### [Crowding_4-3] - RTCI ratio between 1st vs. 2nd run (equal 1 for scenarios 3 and 4, 0 otherwise)
#### [Crowding_4-2] - RTCI ratio between 1st vs. 2nd run (equal 1 for scenarios 5 and 6, 0 otherwise)

In [44]:
df_analysis

,pass_ID,scenario,time_crit,JRT,WT,commuter,age_50_65,age_65plus,Crowding_3_2,Crowding_4_3,Crowding_4_2,output_choice
index,,,,,,,,,,,,
0,1,1,1,20,10,0,0,1,1,0,0,0
0,1,2,1,20,5,0,0,1,1,0,0,1
0,1,3,1,20,10,0,0,1,0,1,0,0
0,1,4,1,20,5,0,0,1,0,1,0,1
0,1,5,1,20,10,0,0,1,0,0,1,0
0,1,6,1,20,5,0,0,1,0,0,1,1
1,2,1,1,15,10,1,0,0,1,0,0,0
1,2,2,1,15,5,1,0,0,1,0,0,0
1,2,3,1,15,10,1,0,0,0,1,0,0


In [45]:
df_analysis.sum()

pass_ID          420750
scenario           7854
time_crit           996
JRT               49110
WT                16830
commuter           1776
age_50_65           108
age_65plus          210
Crowding_3_2        748
Crowding_4_3        748
Crowding_4_2        748
output_choice      1038
dtype: int64

In [46]:
df_analysis.describe()

,pass_ID,scenario,time_crit,JRT,WT,commuter,age_50_65,age_65plus,Crowding_3_2,Crowding_4_3,Crowding_4_2,output_choice
count,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000,2244.000000
mean,187.500000,3.500000,0.443850,21.885027,7.500000,0.791444,0.048128,0.093583,0.333333,0.333333,0.333333,0.462567
std,107.988179,1.708206,0.496948,11.847228,2.500557,0.406367,0.214085,0.291312,0.471510,0.471510,0.471510,0.498708
min,1.000000,1.000000,0.000000,2.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,94.000000,2.000000,0.000000,15.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,187.500000,3.500000,0.000000,20.000000,7.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,281.000000,5.000000,1.000000,30.000000,10.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,374.000000,6.000000,1.000000,90.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# BIOGEME model estimation

## Model 01 - general MNL with all (relevant) attributes
### Utility (dep1) = 0
### Utility (dep2) = booleans (s1... s7) + travel times (wt, jrt)
### Utility (dep2) = [B_timecrit] * s1 + [B_crowding_3-2] * s2 + [B_crowding_4-3] * s3 + [B_crowding_4-2] * s4 + [B_commuter] * s5 + [B_age_50-65] * s6 + [B_age_65plus] * s7
###                        + [B_wt] * WT + [B_jrt] * JRT

In [47]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis)

In [48]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_timecrit = Beta('BETA_timecrit',0,None,None,0)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_commuter = Beta('BETA_commuter',0, None,None,0)
BETA_AGE_50_65 = Beta('BETA_AGE_50_65',0,None,None,0)
BETA_AGE_65 = Beta('BETA_AGE_65',0,None,None,0)
BETA_IVT = Beta('BETA_IVT',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = time_crit
s2 = Crowding_3_2 
s3 = Crowding_4_3
s4 = Crowding_4_2
s5 = commuter
s6 = age_50_65
s7 = age_65plus
ivt = JRT 
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_timecrit * s1 + \
     BETA_CR32 * s2 + \
     BETA_CR43 * s3 + \
     BETA_CR42 * s4 + \
     BETA_commuter * s5 + \
     BETA_AGE_50_65 * s6 + \
     BETA_AGE_65 * s7 + \
     BETA_IVT * ivt + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

## Model 01 estimation results:

In [49]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_001_general_MNL"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_AGE_50_65=	0.698
BETA_AGE_65=	1.98
BETA_CR32=	0.451
BETA_CR42=	2.78
BETA_CR43=	2.69
BETA_IVT=	0.0255
BETA_WT=	-0.298
BETA_commuter=	-0.166
BETA_timecrit=	-1.39
                   Value   Std err     t-test       p-value  Rob. Std err  \
BETA_AGE_50_65  0.697596  0.243663   2.862954  4.197112e-03      0.250783   
BETA_AGE_65     1.978093  0.241584   8.188016  2.220446e-16      0.251962   
BETA_CR32       0.451369  0.237336   1.901815  5.719530e-02      0.239880   
BETA_CR42       2.775450  0.249986  11.102417  0.000000e+00      0.249550   
BETA_CR43       2.688573  0.248856  10.803747  0.000000e+00      0.248277   
BETA_IVT        0.025472  0.004488   5.675468  1.383102e-08      0.004992   
BETA_WT        -0.297578  0.021836 -13.627643  0.000000e+00      0.021828   
BETA_commuter  -0.165975  0.143881  -1.153553  2.486833e-01      0.143810   
BETA_timecrit  -1.387505  0.110454 -12.561848  0.000000e+00      0.111135   

                Rob. t-test  Rob. p-value  
BETA_AGE_50_65     2

## Model 02 - willingness-to-wait, time-critical trips only
### Utility (dep1) = 0
### Utility (dep2) = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_crowding_4-2] * s3 + [B_wt] * WT

In [50]:
# filter time-critical only records
df_analysis_timecrit = df_analysis[df_analysis['time_crit'] == 1]

In [51]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis_timecrit)

In [52]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR32 * s1 + \
     BETA_CR43 * s2 + \
     BETA_CR42 * s3 + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

## Model 02 estimation results:

In [53]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_002_MNL_timecrit_trips"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR32=	-0.19
BETA_CR42=	1.91
BETA_CR43=	1.84
BETA_WT=	-0.303
              Value   Std err    t-test       p-value  Rob. Std err  \
BETA_CR32 -0.189619  0.271892 -0.697407  4.855484e-01      0.272910   
BETA_CR42  1.906383  0.257995  7.389217  1.476597e-13      0.257790   
BETA_CR43  1.835399  0.256520  7.155006  8.366641e-13      0.255936   
BETA_WT   -0.302536  0.031900 -9.483767  0.000000e+00      0.031850   

           Rob. t-test  Rob. p-value  
BETA_CR32    -0.694807  4.871763e-01  
BETA_CR42     7.395106  1.412204e-13  
BETA_CR43     7.171313  7.427392e-13  
BETA_WT      -9.498751  0.000000e+00  


## Model 03 - willingness-to-wait, non-time-critical trips only
### Utility (dep1) = 0
### Utility (dep2) = [B_crowding_3-2] * s1 + [B_crowding_4-3] * s2 + [B_crowding_4-2] * s3 + [B_wt] * WT

In [54]:
# filter time-critical only records
df_analysis_nontimecrit = df_analysis[df_analysis['time_crit'] == 0]

In [55]:
# create Biogeme database
database = db.Database("RTCI_SP_surveys",df_analysis_nontimecrit)

In [56]:
# coefficients
ASC_FIRST = Beta('ASC_FIRST',0,None,None,1)
BETA_CR32 = Beta('BETA_CR32',0,None,None,0)
BETA_CR43 = Beta('BETA_CR43',0,None,None,0)
BETA_CR42 = Beta('BETA_CR42',0,None,None,0)
BETA_WT = Beta('BETA_WT',0,None,None,0)

# variables
s1 = Crowding_3_2 
s2 = Crowding_4_3
s3 = Crowding_4_2
wt = WT 

# utilities
V1 = ASC_FIRST

V2 = BETA_CR32 * s1 + \
     BETA_CR43 * s2 + \
     BETA_CR42 * s3 + \
     BETA_WT * wt

# Associate utility functions with the numbering of alternatives
V = {0: V1,
     1: V2}

# Associate the availability conditions with the alternatives
av = {0: 1,
      1: 1}

## Model 03 estimation results:

In [57]:
logprob = bioLogLogit(V,av,output_choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_003_MNL_nontimecrit_trips"

results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

BETA_CR32=	0.951
BETA_CR42=	3.06
BETA_CR43=	2.97
BETA_WT=	-0.256
              Value   Std err     t-test   p-value  Rob. Std err  Rob. t-test  \
BETA_CR32  0.951277  0.218555   4.352582  0.000013      0.217299     4.377737   
BETA_CR42  3.063423  0.255616  11.984491  0.000000      0.253482    12.085379   
BETA_CR43  2.971007  0.253453  11.722139  0.000000      0.253032    11.741647   
BETA_WT   -0.255532  0.027511  -9.288240  0.000000      0.027496    -9.293532   

           Rob. p-value  
BETA_CR32      0.000012  
BETA_CR42      0.000000  
BETA_CR43      0.000000  
BETA_WT        0.000000  
